<span style="display:block;text-align:center;margin-right:105px"><img src="../../media/logos/logo-vertical.png" width="200"/></span>

# Section 6: Advanced Simulation Methods

---

## Table of Contents
* [cadCAD Simulation Methods](#cadCAD-Simulation-Methods)
  1. [Monte Carlo Method](#1.-Monte-Carlo-Method)
  2. [Parameter Sweeps](#2.-Parameter-Sweeps)
  3. [A/B Testing](#3.-A/B-Testing)
* [cadCAD Simulation Methods Overview](#cadCAD-Simulation-Methods-Overview)
* [Data Manipulation & Analysis](#Data-Manipulation-&-Analysis)

---

\begin{align}
\large population_t &\large= population_{t-1} + {\Delta population} \quad \textrm{(sheep)} \tag{1} \\
\large food_t &\large= food_{t-1} + {\Delta food} \quad \textrm{(tons of grass)} \tag{2}
\end{align}

where the rate of change ($\Delta$) is:
\begin{align}
\large {\Delta population} &\large= \alpha * food_{t-1} - \epsilon * population_{t-1} \quad \textrm{(sheep/month)} \\
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma \quad \textrm{(tons of grass/month)}
\end{align}

where:

$
\begin{align}
\alpha: \quad &\textrm{'reproduction_rate'}\\
\epsilon: \quad &\textrm{'death_rate'}\\
\beta: \quad &\textrm{'consumption_rate'}\\
\gamma: \quad &\textrm{'growth_rate'}\\
\end{align}
$

* A population consumes a food source, and reproduces at a rate proportional to the food source $\alpha$ (alpha), and dies at a rate proportional to the population size $\epsilon$ (epsilon).
* The food source is consumed at a rate proportional to the population $\beta$ (beta), and grows at a constant rate $\gamma$ (gamma).

<center>
<img src="./images/s6-differential-spec-ecosystem-final.png"
     alt="Diff spec"
     style="width: 60%" />
</center>

Import and run the existing ecosystem model code from section 5:

In [1]:
import math
import pandas as pd
import plotly

pd.options.plotting.backend = "plotly"

from cadCAD.configuration.utils import config_sim

from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [2]:
from cadCAD.configuration import Experiment

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()

# cadCAD Simulation Methods

1. [Monte Carlo Method](#1.-Monte-Carlo-Method)
2. [Parameter Sweeps](#2.-Parameter-Sweeps)
3. [A/B testing](#3.-A/B-Testing)

## 1. Monte Carlo Method

> Monte Carlo methods, or Monte Carlo experiments, are a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle.

Source: https://en.wikipedia.org/wiki/Monte_Carlo_method

### Non-determinism

> In computer science, a nondeterministic algorithm is an algorithm that, even for the same input, can exhibit different behaviors on different runs, as opposed to a deterministic algorithm.

Source: https://en.wikipedia.org/wiki/Nondeterministic_algorithm

\begin{align}
\large {\Delta food} &\large= -\beta * population_{t-1} + \gamma * rand() \quad \textrm{(tons of grass/month)} \\
\end{align}


* Addition of a random food growth rate, using `rand()`.

In [3]:
from numpy import random

In [4]:
random.rand()

0.8467436528420059

In [5]:
# Setting a seed so that we can reproduce the experiment
random.seed(1234)

In [6]:
# With a given seed, generates a random number from a given set
[
    random.rand(),
    random.rand()
]

[0.1915194503788923, 0.6221087710398319]

In [7]:
random.seed(1234)
[
    random.rand(),
    random.rand()
]

[0.1915194503788923, 0.6221087710398319]

In [8]:
random.seed(None)

In [9]:
seeds = [
    random.RandomState(1),
    random.RandomState(2),
    random.RandomState(3),
]

In [10]:
[
    seeds[0].rand(),
    seeds[1].rand(),
    seeds[2].rand(),
]

[0.417022004702574, 0.43599490214200376, 0.5507979025745755]

In [11]:
MONTE_CARLO_RUNS = 3

seeds = [random.RandomState(i) for i in range(MONTE_CARLO_RUNS)]
seeds

[RandomState(MT19937) at 0x23142BAB840,
 RandomState(MT19937) at 0x23142BAB940,
 RandomState(MT19937) at 0x23142BABA40]

In [12]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'reproduction_rate': [0.3],
    'death_rate': [0.03],
    'consumption_rate': [0.03],
    'growth_rate': [30.0],
}

In [13]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food']
    return 'food', max(food, 0)

In [14]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_death(params, substep, state_history, previous_state):
    population_death = params['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_growth(params, substep, state_history, previous_state):
    run = previous_state['run']
    food_growth = params['growth_rate'] * seeds[run - 1].rand()
    return {'delta_food': food_growth}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}

In [15]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [16]:
SIMULATION_TIMESTEPS = 500

In [17]:
sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config,
)

In [18]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (500, 4, 3, 2)
Execution Method: local_simulations
SimIDs   : [0, 0, 0]
SubsetIDs: [0, 0, 0]
Ns       : [0, 1, 2]
ExpIDs   : [0, 0, 0]
Execution Mode: parallelized
Total execution time: 0.08s


In [19]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,population,food,simulation,subset,run,substep,timestep
0,50,1000.000000,0,0,1,0,0
1,349,1014.964405,0,0,1,1,1
2,644,1025.950086,0,0,1,1,2
3,933,1024.712987,0,0,1,1,3
4,1213,1013.069483,0,0,1,1,4
...,...,...,...,...,...,...,...
1498,649,88.611704,0,0,3,1,496
1499,657,70.673419,0,0,3,1,497
1500,659,70.872657,0,0,3,1,498
1501,661,62.072474,0,0,3,1,499


In [20]:
df = simulation_result.copy()
df = df[df.simulation == 0]
df

,population,food,simulation,subset,run,substep,timestep
0,50,1000.000000,0,0,1,0,0
1,349,1014.964405,0,0,1,1,1
2,644,1025.950086,0,0,1,1,2
3,933,1024.712987,0,0,1,1,3
4,1213,1013.069483,0,0,1,1,4
...,...,...,...,...,...,...,...
1498,649,88.611704,0,0,3,1,496
1499,657,70.673419,0,0,3,1,497
1500,659,70.872657,0,0,3,1,498
1501,661,62.072474,0,0,3,1,499


In [21]:
df[df.run == 1].head()

,population,food,simulation,subset,run,substep,timestep
0,50,1000.000000,0,0,1,0,0
1,349,1014.964405,0,0,1,1,1
2,644,1025.950086,0,0,1,1,2
3,933,1024.712987,0,0,1,1,3
4,1213,1013.069483,0,0,1,1,4


In [22]:
df[df.run == 2].head()

,population,food,simulation,subset,run,substep,timestep
501,50,1000.000000,0,0,2,0,0
502,349,1011.010660,0,0,2,1,1
503,642,1022.150395,0,0,2,1,2
504,930,1002.893826,0,0,2,1,3
505,1203,984.063803,0,0,2,1,4


In [23]:
df[df.run == 3].head()

,population,food,simulation,subset,run,substep,timestep
1002,50,1000.000000,0,0,3,0,0
1003,349,1011.579847,0,0,3,1,1
1004,643,1001.887634,0,0,3,1,2
1005,925,999.087508,0,0,3,1,3
1006,1197,984.397180,0,0,3,1,4


### Simulation Analysis

In [24]:
import plotly.express as px

px.line(df, x='timestep', y=['population', 'food'], facet_row='run', height=800, width=1000)

## 2. Parameter Sweeps

In [25]:
dictionary = {
    'reproduction_rate': [0.1, 0.2],
    'death_rate': [0.01, 0.02]
}
dictionary['reproduction_rate'][0]

0.1

In [26]:
nested_dictionary = {
    'population': [
        {'reproduction_rate': 0.1, 'death_rate': 0.01},
        {'reproduction_rate': 0.2, 'death_rate': 0.02},
    ]
}
nested_dictionary['population'][0]['reproduction_rate']

0.1

In [27]:
initial_state = {
    'population': 50, # number of sheep
    'food': 1000 # tons of grass
}

system_params = {
    'population': [
        {'reproduction_rate': 0.1, 'death_rate': 0.01},
        {'reproduction_rate': 0.2, 'death_rate': 0.02},
        {'reproduction_rate': 0.3, 'death_rate': 0.03}
    ],
    'food': [
        {'consumption_rate': 0.01, 'growth_rate': 10.0},
        {'consumption_rate': 0.02, 'growth_rate': 20.0},
        {'consumption_rate': 0.03, 'growth_rate': 30.0}
    ]
}

In [28]:
{
    'population': system_params['population'][0],
    'food': system_params['food'][0]
}

{'population': {'reproduction_rate': 0.1, 'death_rate': 0.01},
 'food': {'consumption_rate': 0.01, 'growth_rate': 10.0}}

In [29]:
{
    'population': system_params['population'][1],
    'food': system_params['food'][1]
}

{'population': {'reproduction_rate': 0.2, 'death_rate': 0.02},
 'food': {'consumption_rate': 0.02, 'growth_rate': 20.0}}

In [30]:
def s_population(params, substep, state_history, previous_state, policy_input):
    population = previous_state['population'] + policy_input['delta_population'] 
    return 'population', max(math.ceil(population), 0)

def s_food(params, substep, state_history, previous_state, policy_input):
    food = previous_state['food'] + policy_input['delta_food']
    return 'food', max(food, 0)

In [31]:
def p_reproduction(params, substep, state_history, previous_state):
    population_reproduction = params['population']['reproduction_rate'] * previous_state['food']
    return {'delta_population': population_reproduction}

def p_death(params, substep, state_history, previous_state):
    population_death = params['population']['death_rate'] * previous_state['population']
    return {'delta_population': -population_death}

def p_consumption(params, substep, state_history, previous_state):
    food_consumption = params['food']['consumption_rate'] * previous_state['population']
    return {'delta_food': -food_consumption}
    
def p_growth(params, substep, state_history, previous_state):
    food_growth = params['food']['growth_rate']
    return {'delta_food': food_growth}

In [32]:
partial_state_update_blocks = [
    {
        'policies': {
            'reproduction': p_reproduction,
            'death': p_death,
            'consumption': p_consumption,
            'growth': p_growth
        },
        'variables': {
            'population': s_population,
            'food': s_food
        }
    }
]

In [33]:
MONTE_CARLO_RUNS = 1
SIMULATION_TIMESTEPS = 500

sim_config = config_sim({
    'N': MONTE_CARLO_RUNS,
    'T': range(SIMULATION_TIMESTEPS),
    'M': system_params
})

In [34]:
from pprint import pprint

print('system_params')
pprint(system_params)
print(' ')
print('sim_config')
pprint(sim_config)

system_params
{'food': [{'consumption_rate': 0.01, 'growth_rate': 10.0},
          {'consumption_rate': 0.02, 'growth_rate': 20.0},
          {'consumption_rate': 0.03, 'growth_rate': 30.0}],
 'population': [{'death_rate': 0.01, 'reproduction_rate': 0.1},
                {'death_rate': 0.02, 'reproduction_rate': 0.2},
                {'death_rate': 0.03, 'reproduction_rate': 0.3}]}
 
sim_config
[{'M': {'food': {'consumption_rate': 0.01, 'growth_rate': 10.0},
        'population': {'death_rate': 0.01, 'reproduction_rate': 0.1}},
  'N': 1,
  'T': range(0, 500)},
 {'M': {'food': {'consumption_rate': 0.02, 'growth_rate': 20.0},
        'population': {'death_rate': 0.02, 'reproduction_rate': 0.2}},
  'N': 1,
  'T': range(0, 500)},
 {'M': {'food': {'consumption_rate': 0.03, 'growth_rate': 30.0},
        'population': {'death_rate': 0.03, 'reproduction_rate': 0.3}},
  'N': 1,
  'T': range(0, 500)}]


In [35]:
experiment.append_configs(
    initial_state = initial_state,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [36]:
exec_mode = ExecutionMode()
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 2
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (500, 4, 3, 2)
Execution Method: local_simulations
SimIDs   : [0, 0, 0, 1, 1, 1]
SubsetIDs: [0, 0, 0, 0, 1, 2]
Ns       : [0, 1, 2, 0, 1, 2]
ExpIDs   : [0, 0, 0, 1, 1, 1]
Execution Mode: parallelized
Total execution time: 0.18s


In [37]:
simulation_result = pd.DataFrame(raw_result)
simulation_result

,population,food,simulation,subset,run,substep,timestep
0,50,1000.000000,0,0,1,0,0
1,349,1007.811425,0,0,1,1,1
2,641,1008.532471,0,0,1,1,2
3,925,1005.051584,0,0,1,1,3
4,1199,999.819435,0,0,1,1,4
...,...,...,...,...,...,...,...
3001,1000,99.960000,1,2,3,1,496
3002,1000,99.960000,1,2,3,1,497
3003,1000,99.960000,1,2,3,1,498
3004,1000,99.960000,1,2,3,1,499


In [38]:
df = simulation_result.copy()
df = df[df.simulation == 1]
df

,population,food,simulation,subset,run,substep,timestep
1503,50,1000.00,1,0,1,0,0
1504,150,1009.50,1,0,1,1,1
1505,250,1018.00,1,0,1,1,2
1506,350,1025.50,1,0,1,1,3
1507,450,1032.00,1,0,1,1,4
...,...,...,...,...,...,...,...
3001,1000,99.96,1,2,3,1,496
3002,1000,99.96,1,2,3,1,497
3003,1000,99.96,1,2,3,1,498
3004,1000,99.96,1,2,3,1,499


### Simulation Analysis

In [39]:
import plotly.express as px

px.line(df, x='timestep', y=['population', 'food'], facet_row='subset', height=800, width=1000)

## 3. A/B Testing

In [40]:
df = simulation_result.copy()
df

,population,food,simulation,subset,run,substep,timestep
0,50,1000.000000,0,0,1,0,0
1,349,1007.811425,0,0,1,1,1
2,641,1008.532471,0,0,1,1,2
3,925,1005.051584,0,0,1,1,3
4,1199,999.819435,0,0,1,1,4
...,...,...,...,...,...,...,...
3001,1000,99.960000,1,2,3,1,496
3002,1000,99.960000,1,2,3,1,497
3003,1000,99.960000,1,2,3,1,498
3004,1000,99.960000,1,2,3,1,499


In [41]:
import plotly.express as px

fig = px.line(
    df,
    x='timestep',
    y=['population', 'food'],
    facet_row='simulation',
    facet_col='run',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

# cadCAD Simulation Methods Overview

<center>
<img src="./images/cadcad-simulation-methods.png"
     alt="Simulation methods"
     style="width: 600px;" />
</center>

# Data Manipulation & Analysis

In [42]:
df = simulation_result.copy()
df

,population,food,simulation,subset,run,substep,timestep
0,50,1000.000000,0,0,1,0,0
1,349,1007.811425,0,0,1,1,1
2,641,1008.532471,0,0,1,1,2
3,925,1005.051584,0,0,1,1,3
4,1199,999.819435,0,0,1,1,4
...,...,...,...,...,...,...,...
3001,1000,99.960000,1,2,3,1,496
3002,1000,99.960000,1,2,3,1,497
3003,1000,99.960000,1,2,3,1,498
3004,1000,99.960000,1,2,3,1,499


In [43]:
df = df.query("simulation == 1 and run == 2")
df

,population,food,simulation,subset,run,substep,timestep
2004,50,1000.00,1,1,2,0,0
2005,249,1019.00,1,1,2,1,1
2006,448,1034.02,1,1,2,1,2
2007,646,1045.06,1,1,2,1,3
2008,843,1052.14,1,1,2,1,4
...,...,...,...,...,...,...,...
2500,994,95.04,1,1,2,1,496
2501,994,95.16,1,1,2,1,497
2502,994,95.28,1,1,2,1,498
2503,994,95.40,1,1,2,1,499


In [44]:
fig = px.scatter(
    df[df.timestep >= 80],
    x='population',
    y='food',
    color='timestep'
)
fig.show()

<br/><br/><br/>
# Well done!
You're now well on your way to being a cadCAD system modeller!
<br/><br/><br/><br/>